In [4]:
import struct
import numpy as np

In [5]:
def checkdata(data):
    print(f'title={data['Title']}')
    # print('N0={data['N0']}', ' ', data['NN'], ' ', data['NFreq1'], ' ', data['nout'],' ', data['Dt'], ' ', data['NGline'])
    print(f'Nx={data['Nx']}, Ny={data['Ny']}, Nz={data['Nz']}, Ni={data['Ni']}, Nj={data['Nj']}, Nk={data['Nk']}, N0={data['N0']}')
    print(f'NN={data['NN']}, NFreq1={data['NFreq1']}, NFreq2={data['NFreq2']}, NFeed={data['NFeed']}, NPoint={data['NPoint']}')
    print(f'Niter={data['Niter']}') 
    


In [44]:
def read_ofd_out(file_path):
    """
    OpenFDTDの出力ファイル 'ofd.out' をPythonで読み込む関数。

    Args:
        file_path (str): 読み込むofd.outファイルのパス。

    Returns:
        dict: 読み込んだデータを格納した辞書。
    """
    data = {}
    try:
        with open(file_path, 'rb') as fp:
            # ヘッダー情報の読み込み
            data['Title'] = fp.read(256).decode('utf-8').strip('\x00')
            header_fmt = '<iiiiiiiqiiiiiiiidii'
            header_size = struct.calcsize(header_fmt)
            header_bytes = fp.read(header_size)
            (data['Nx'], data['Ny'], data['Nz'], data['Ni'], data['Nj'], data['Nk'], data['N0'],
             data['NN'], data['NFreq1'], data['NFreq2'], data['NFeed'], data['NPoint'],
             data['Niter'], data['Ntime'], data['maxiter'], data['nout'], data['Dt'],
             data['NGline'], data['IPlanewave']) = struct.unpack(header_fmt, header_bytes)

#            checkdata(data)

            # Planewave構造体の読み込み
            planewave_fmt = '<dd'
            planewave_size = struct.calcsize(planewave_fmt)
            planewave_bytes = fp.read(planewave_size)

            (data['Planewave_theta'], data['Planewave_phi']) = struct.unpack(planewave_fmt, planewave_bytes)
            
            data['Planewave_ei'] = np.fromfile(fp, dtype=np.float64, count=3)
            data['Planewave_hi'] = np.fromfile(fp, dtype=np.float64, count=3)
            data['Planewave_ri'] = np.fromfile(fp, dtype=np.float64, count=3)
            data['Planewave_r0'] = np.fromfile(fp, dtype=np.float64, count=3)
 
            planewave_fmt = '<di'
            planewave_size = struct.calcsize(planewave_fmt)
            planewave_bytes = fp.read(planewave_size)
            (data['Planewave_ai'], data['Planewave_pol']) = struct.unpack(planewave_fmt, planewave_bytes)

            temp = fp.read(4)

            # 配列データの読み込み
            data['Xn'] = np.fromfile(fp, dtype=np.float64, count=data['Nx'] + 1)
            data['Yn'] = np.fromfile(fp, dtype=np.float64, count=data['Ny'] + 1)
            data['Zn'] = np.fromfile(fp, dtype=np.float64, count=data['Nz'] + 1)
            data['Xc'] = np.fromfile(fp, dtype=np.float64, count=data['Nx'])
            data['Yc'] = np.fromfile(fp, dtype=np.float64, count=data['Ny'])
            data['Zc'] = np.fromfile(fp, dtype=np.float64, count=data['Nz'])
            data['Eiter'] = np.fromfile(fp, dtype=np.float64, count=data['Niter'])
            data['Hiter'] = np.fromfile(fp, dtype=np.float64, count=data['Niter'])
            data['VFeed'] = np.fromfile(fp, dtype=np.float64, count=data['NFeed'] * (data['maxiter'] + 1))
            data['IFeed'] = np.fromfile(fp, dtype=np.float64, count=data['NFeed'] * (data['maxiter'] + 1))
            data['VPoint'] = np.fromfile(fp, dtype=np.float64, count=data['NPoint'] * (data['maxiter'] + 1))
            data['Freq1'] = np.fromfile(fp, dtype=np.float64, count=data['NFreq1'])
            data['Freq2'] = np.fromfile(fp, dtype=np.float64, count=data['NFreq2'])

            # feed_t構造体とd_complex_t構造体の読み込みは、`struct`モジュールを使ってループで処理することも可能
            # ここではシンプルに、`np.fromfile`で読み込み、後で整形するアプローチを示す
            # Zin (d_complex_t) の読み込み
            zin_count = data['NFeed'] * data['NFreq1']
            zin_data = np.fromfile(fp, dtype=np.float64, count=zin_count * 2)
            data['Zin'] = zin_data.view(dtype=np.complex128)

            data['Ref'] = np.fromfile(fp, dtype=np.float64, count=data['NFeed'] * data['NFreq1'])
            
            pin_count = data['NFeed'] * data['NFreq2']
            data['Pin0'] = np.fromfile(fp, dtype=np.float64, count=pin_count)
            data['Pin1'] = np.fromfile(fp, dtype=np.float64, count=pin_count)

            # Spara (d_complex_t) の読み込み
            spara_count = data['NPoint'] * data['NFreq1']
            spara_data = np.fromfile(fp, dtype=np.float64, count=spara_count * 2)
            data['Spara'] = spara_data.view(dtype=np.complex128)

            # Glineの読み込み
            data['Gline'] = np.fromfile(fp, dtype=np.float64, count=data['NGline'] * 2 * 3).reshape(data['NGline'], 2, 3)

            # ...以下、Cコードの fread に対応する部分を記述...
            # SurfaceとSurfaceEx,Ey,Ezなどの読み込み
            data['NSurface'] = struct.unpack('i', fp.read(4))[0]

            # print(f'Gline={data['Gline']}')
            # print(f'NSuaface={data['NSurface']}')

            # surface_t 構造体 (ここでは仮のサイズで実装)
            surface_fmt = '<ddddddd' # 適当な例
            surface_count = data['NSurface']
            surface_data = fp.read(surface_count * struct.calcsize(surface_fmt))
            data['Surface'] = [struct.unpack(surface_fmt, surface_data[i:i + struct.calcsize(surface_fmt)]) for i in range(0, len(surface_data), struct.calcsize(surface_fmt))]

            NSurface = data['NSurface']
            data['surface_nx'] = np.zeros(NSurface, dtype=np.float64)
            data['surface_ny'] = np.zeros(NSurface, dtype=np.float64)
            data['surface_nz'] = np.zeros(NSurface, dtype=np.float64)
            data['surface_x'] = np.zeros(NSurface, dtype=np.float64)
            data['surface_y'] = np.zeros(NSurface, dtype=np.float64)
            data['surface_z'] = np.zeros(NSurface, dtype=np.float64)
            data['surface_ds'] = np.zeros(NSurface, dtype=np.float64)
            surface_fmt = '<d'
            for i in range(NSurface):
                surface_data = fp.read(struct.calcsize(surface_fmt))
                (data['surface_nx'][i],) = struct.unpack(surface_fmt, surface_data)
                surface_data = fp.read(struct.calcsize(surface_fmt))
                (data['surface_ny'][i],) = struct.unpack(surface_fmt, surface_data)
                surface_data = fp.read(struct.calcsize(surface_fmt))
                (data['surface_nz'][i],) = struct.unpack(surface_fmt, surface_data)
                surface_data = fp.read(struct.calcsize(surface_fmt))
                (data['surface_x'][i],) = struct.unpack(surface_fmt, surface_data)
                surface_data = fp.read(struct.calcsize(surface_fmt))
                (data['surface_y'][i],) = struct.unpack(surface_fmt, surface_data)
                surface_data = fp.read(struct.calcsize(surface_fmt))
                (data['surface_z'][i],) = struct.unpack(surface_fmt, surface_data)
                surface_data = fp.read(struct.calcsize(surface_fmt))
                (data['surface_ds'][i],) = struct.unpack(surface_fmt, surface_data)

            print(data['surface_ds'][NSurface-1])    
            
            # SurfaceExなどの読み込み
            data['SurfaceEx'] = [np.fromfile(fp, dtype=np.complex128, count=data['NSurface']) for _ in range(data['NFreq2'])]
            data['SurfaceEy'] = [np.fromfile(fp, dtype=np.complex128, count=data['NSurface']) for _ in range(data['NFreq2'])]
            data['SurfaceEz'] = [np.fromfile(fp, dtype=np.complex128, count=data['NSurface']) for _ in range(data['NFreq2'])]
            data['SurfaceHx'] = [np.fromfile(fp, dtype=np.complex128, count=data['NSurface']) for _ in range(data['NFreq2'])]
            data['SurfaceHy'] = [np.fromfile(fp, dtype=np.complex128, count=data['NSurface']) for _ in range(data['NFreq2'])]
            data['SurfaceHz'] = [np.fromfile(fp, dtype=np.complex128, count=data['NSurface']) for _ in range(data['NFreq2'])]

            # cEx_rなどの読み込み
            data['cEx_r'] = np.fromfile(fp, dtype=np.float32, count=data['NN'] * data['NFreq2'])
            data['cEx_i'] = np.fromfile(fp, dtype=np.float32, count=data['NN'] * data['NFreq2'])
            data['cEy_r'] = np.fromfile(fp, dtype=np.float32, count=data['NN'] * data['NFreq2'])
            data['cEy_i'] = np.fromfile(fp, dtype=np.float32, count=data['NN'] * data['NFreq2'])
            data['cEz_r'] = np.fromfile(fp, dtype=np.float32, count=data['NN'] * data['NFreq2'])
            data['cEz_i'] = np.fromfile(fp, dtype=np.float32, count=data['NN'] * data['NFreq2'])
            data['cHx_r'] = np.fromfile(fp, dtype=np.float32, count=data['NN'] * data['NFreq2'])
            data['cHx_i'] = np.fromfile(fp, dtype=np.float32, count=data['NN'] * data['NFreq2'])
            data['cHy_r'] = np.fromfile(fp, dtype=np.float32, count=data['NN'] * data['NFreq2'])
            data['cHy_i'] = np.fromfile(fp, dtype=np.float32, count=data['NN'] * data['NFreq2'])
            data['cHz_r'] = np.fromfile(fp, dtype=np.float32, count=data['NN'] * data['NFreq2'])
            data['cHz_i'] = np.fromfile(fp, dtype=np.float32, count=data['NN'] * data['NFreq2'])

            # 最終的なサイズチェック
            final_size = struct.unpack('Q', fp.read(8))[0]

            print(final_size)
            print(fp.tell())
            if fp.tell() != final_size:
                 print(f"警告: ファイルサイズが期待値と一致しません。読み込みが不完全な可能性があります。")

            return data

    except FileNotFoundError:
        print(f"エラー: ファイル '{file_path}' が見つかりません。")
        return None
    except struct.error as e:
        print(f"エラー: バイナリデータの読み込みに失敗しました。ファイル形式が不正である可能性があります。詳細: {e}")
        return None



In [45]:
# 使用例
file_name = '../ofd.out'
ofd_data = read_ofd_out(file_name)

if ofd_data:
    
    print("ファイルから読み込んだ主なヘッダー情報:")
    print(f"Title: {ofd_data['Title']}")
    print(f"Nx: {ofd_data['Nx']}, Ny: {ofd_data['Ny']}, Nz: {ofd_data['Nz']}")
    print(f"NN: {ofd_data['NN']}")
    print(f"VFeedの最初の5つの値: {ofd_data['VFeed'][:5]}")

0.005934965713095153
エラー: バイナリデータの読み込みに失敗しました。ファイル形式が不正である可能性があります。詳細: unpack requires a buffer of 8 bytes


In [19]:
tmp=[]

In [20]:
tmp.append(1.0)

In [30]:
tmp={}
tmp['data']=[]
tmp['data'].append(2.0)

In [31]:
tmp

{'data': [2.0]}

In [26]:
tmp

NameError: name 'tmp' is not defined